# Data Processing

In [10]:
from glob import glob
from PIL import Image
from pathlib import Path
from tqdm import tqdm

import numpy as np

In [11]:
food_product = 'tacos'
filepath = Path('data/images') / Path(food_product)
files = glob(str(filepath) + "/*.jpg")
print(f"Number of Files: {len(files)}")

Number of Files: 1000


In [13]:
resize_value = (128, 128)
images = []

for file in tqdm(files):
    image = Image.open(file)
    image = image.resize(resize_value)
    image = np.array(image)
    
    if(len(image.shape) != 3):
        image = np.stacK((image, image, image), axis = 2)
        images.append(image)
    else:
        images.append(image)

images = np.array(images)

100%|██████████| 1000/1000 [00:03<00:00, 323.85it/s]


In [14]:
save_path = Path('data/numpy') / Path(food_product + '.npy')
np.save(str(save_path), images)

In [31]:
# Model Training
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.autograd import Variable

%matplotlib inline
import matplotlib.pyplot as plt

from scripts.dcgan import Generator, Discriminator, weights_init_normal

In [32]:
class Dataloader:
    def __init__(self, data):
        self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return torch.from_numpy(self.data[idx])

In [33]:
opt = {
    "n_epochs": 1,
    "batch_size": 64,
    "lr_g": 0.001,
    "lr_d": 0.0001,
    "b1": 0.5,
    "b2":0.999,
    "latent_dim": 500,
    "img_size": 128,
    "channels": 3,
    "sample_interval": 400
}

In [34]:
save_path = Path('drive/MyDrive/Food Images - 128/tacos.npy')
data = np.load()

cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

loss = nn.BCELoss()

# Define the dataloader
dataloader = torch.utils.data.DataLoader(Dataloader(images), batch_size=64, shuffle=True, drop_last=True)

# Define the models.
generator = Generator(opt)
discriminator = Discriminator(opt)

# Convert to CUDA if available
if cuda:
    generator = generator.cuda()
    discriminator = discriminator.cuda()
    adversarial_loss = loss.cuda()

# Apply weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Initialize optimizers.
opt_g = torch.optim.Adam(generator.parameters(), lr = opt['lr_g'], betas = (opt['b1'], opt['b2']))
opt_d = torch.optim.Adam(discriminator.parameters(), lr = opt['lr_d'], betas = (opt['b1'], opt['b2']))

In [35]:
for epoch in range(opt['n_epochs']):
    for idx, images in enumerate(dataloader):

        # Create the True/False Identifiers for Discriminator
        valid = Variable(Tensor(images.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(images.shape[0], 1).fill_(0.0), requires_grad=False)

        real_imgs = Variable(images.type(Tensor)) / 127.5 - 0.5

        opt_g.zero_grad()
    
        # Generate Images.
        z = Variable(Tensor(np.random.normal(0, 1, (images.shape[0], opt['latent_dim']))))
        gen_imgs = generator(z)

        # Train the Generator
        g_loss = loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        opt_g.step()

        # Train the Discriminator
        opt_d.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = loss(discriminator(real_imgs), valid)
        fake_loss = loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        opt_d.step()

        if idx % 10 == 0:
            print(
                "[Epoch %d/%d]\t[Batch %d/%d]\t[D loss: %f]\t[G loss: %f]"
                % (epoch, opt['n_epochs'], idx, len(dataloader), d_loss.item(), g_loss.item())
            )
        if idx % 50 == 0:
            z_temp = Tensor(np.random.normal(0, 1, (1, opt['latent_dim'])))
            a = generator(z_temp).permute(0, 2, 3, 1).cpu().detach().numpy()
            a = (a[0] / 2) + 0.5
            plt.imshow(a, interpolation='nearest')
            plt.show()